In [10]:
# ============================================================================
# ENVIRONMENT SETUP: Load API Keys & Import Dependencies
# ============================================================================
# We use python-dotenv to securely load API keys from a .env file
# This is a best practice - never hardcode API keys in your notebooks!
# ============================================================================

from dotenv import load_dotenv
import os
import sys
import platform

# Load environment variables from .env file
load_dotenv()

# Add parent directory to path for importing helpers
sys.path.append(os.path.abspath("../.."))

# Import our LLM factory functions
# - get_groq_llm(): Creates a Groq-hosted LLM (fast inference with open-source models)
# - get_openai_llm(): Creates an OpenAI GPT model
# - get_databricks_llm(): Creates a Databricks-hosted LLM
from helpers.utils import get_groq_llm, get_openai_llm, get_databricks_llm

print("✅ Environment variables loaded successfully!")
print(f"📍 Running on: {platform.system()}")

# -----------------------------------------------------------------------------
# Initialize the LLM based on platform or preference
# The choice of LLM affects tool calling capabilities and speed
# -----------------------------------------------------------------------------
if sys.platform == "win32":
    # Windows: Use Groq for fast inference
    llm = get_groq_llm()
elif sys.platform == "darwin":
    # macOS: Use Databricks-hosted Gemini
    llm = get_databricks_llm("databricks-gemini-2-5-pro")  
else:
    # Linux: Default to Groq
    llm = get_groq_llm()

# Print which LLM we're using
if hasattr(llm, 'model_name'):
    print(f"🤖 LLM initialized: {llm.model_name}")
elif hasattr(llm, 'model'):
    print(f"🤖 LLM initialized: {llm.model}")
else:
    print("🤖 LLM initialized successfully")

✅ Environment variables loaded successfully!
📍 Running on: Darwin
🤖 LLM initialized: databricks-gemini-2-5-pro


### Introduction of LangChain

In [11]:
try:
    # Note: llm is already initialized in the setup cell using helper functions
    # The llm object is created based on your platform (Windows: Groq, macOS: Databricks, Linux: Groq)

    # Get the response
    response = llm.invoke('What is the capital of France?')
    print(response.content)
    
    print("-------------------------------------------------")
    
    # Use llm.invoke() with another question
    alt_response = llm.invoke("What is LangChain? Explain in 1 line")
    print(alt_response.content)
except Exception as e:
    print(e)

The capital of France is **Paris**.
-------------------------------------------------
LangChain is a framework that simplifies building applications powered by large language models (LLMs) by connecting them to other data sources and tools.


### First LangChain Project 

A LangChain pipeline is a sequence of steps where inputs are processed to produce outputs. For your first project, you’ll create a basic pipeline that uses an LLM to generate text responses based on user input.



Build a pipeline that: 
* Accepts a user’s question as input.  
* Uses a pre-trained LLM to generate a response.   
* Outputs the generated text.

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [13]:
try:
    
    # Step 1: Create a Prompt Template
    prompt = PromptTemplate(
        input_variables=['topic'],
        template="Provide me details about {topic}?"
    )

    # Step 2: Create a Chain using LCEL (LangChain Expression Language)
    # The | operator chains: prompt -> llm -> output parser
    chain = prompt | llm | StrOutputParser()

    # Step 3: Generate the response
    response = chain.invoke({'topic': 'Generative AI'})
    resp = chain.invoke({'topic': 'Deep Learning'})
    
    print(response)
    print("-------------------------------------------------")
    print(resp)
    
except Exception as e:
    print(e)

Of course! Here is a detailed breakdown of Generative AI, covering what it is, how it works, its applications, key players, and the associated challenges.

### 1. What is Generative AI?

At its core, **Generative AI** is a subfield of artificial intelligence that focuses on **creating new, original content** rather than just analyzing or classifying existing data.

Think of it this way:
*   **Traditional AI (or Discriminative AI)** is like a student who learns to *classify* things. You show it thousands of pictures of cats and dogs, and it learns to tell the difference, labeling a new picture as either "cat" or "dog."
*   **Generative AI** is like a student who, after seeing thousands of pictures of cats, can *draw a brand new, unique picture of a cat* that has never existed before.

This "new content" can be in various forms, including text, images, music, code, and even video.

---

### 2. How Does Generative AI Work?

Generative AI models are trained on massive datasets of existing 

/Users/sourav.banerjee/Documents/Codebases/2. AI ENGINEERING/LangChain_Demystified/.venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='content', input_value=[{'type': 'text', 'text':...we diagnose diseases.'}], input_type=list])
  return self.__pydantic_serializer__.to_python(


### Modify the above project
* After generating the response, summarize it into a shorter version.

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

try:
    
    # Step 1: Create Prompt Templates
    detail_prompt = PromptTemplate(
        input_variables=['topic'],
        template="Provide me details about {topic}?"
    )

    summary_prompt = PromptTemplate(
        input_variables=['text'],
        template="Summarize the following text into a single concise sentence: {text}"
    )

    # Step 2: Create individual chains using LCEL
    answer_chain = detail_prompt | llm | StrOutputParser()
    summary_chain = summary_prompt | llm | StrOutputParser()

    # Step 3: Combine the Chains using LCEL
    # First chain generates details, then passes to summary chain
    pipeline = (
        {"topic": RunnablePassthrough()}
        | answer_chain
        | (lambda text: {"text": text})
        | summary_chain
    )

    # Step 4: Generate the Response
    response = pipeline.invoke('Deep Learning')

    print(response)
        
    
except Exception as e:
    print(e)

/Users/sourav.banerjee/Documents/Codebases/2. AI ENGINEERING/LangChain_Demystified/.venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='content', input_value=[{'type': 'text', 'text':...ot trained carefully.'}], input_type=list])
  return self.__pydantic_serializer__.to_python(


Deep Learning is a machine learning technique that uses multi-layered artificial neural networks, inspired by the human brain, to automatically learn complex patterns and features from vast amounts of data.


### Add a Sentiment Analysis Step at the end

In [15]:
# Create a sentiment analysis prompt
sentiment_prompt = PromptTemplate(
    input_variables=['text'],
    template="Analyze the sentiment of the following text: {text}"
)

# Create the sentiment chain
sentiment_chain = sentiment_prompt | llm | StrOutputParser()

# Combine all three chains: details -> summary -> sentiment
full_pipeline = (
    {"topic": RunnablePassthrough()}
    | answer_chain
    | (lambda text: {"text": text})
    | summary_chain
    | (lambda text: {"text": text})
    | sentiment_chain
)

response = full_pipeline.invoke('Deep Learning')

print(response)

Based on the text provided, the sentiment is **Neutral**.

**Analysis:**

The text is purely informational and descriptive. It defines Deep Learning in a factual, objective manner, explaining its inspiration, what it is, and how it works. There are no words or phrases that express a positive or negative opinion, emotion, or judgment.


### Add a Dynamic User Input Chain

In [0]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Step 1: Create Prompt Template
detail_prompt = PromptTemplate(
    input_variables=['topic'],
    template="Provide me details about {topic}?"
)

# Step 2: Create the chain using LCEL
answer_chain = detail_prompt | llm | StrOutputParser()

while True:
    user_input = input("Enter the topic you want to know more about (or type 'exit' to quit): ")
    
    # Exit the loop if the user types 'exit'
    if user_input.lower() == 'exit':
        print("Exiting the assistant. GoodBye!!")
        break
    
    response = answer_chain.invoke({'topic': user_input})
    print(response)



> Entering new LLMChain chain...
Prompt after formatting:
Provide me details about Deep Learning?

> Finished chain.


Deep learning is a subset of machine learning that utilizes artificial neural networks to learn and make decisions from data. It is a form of artificial intelligence that mimics the way the human brain processes information. 

In deep learning, the neural networks have multiple layers of interconnected nodes that can process and transform data at each layer. The layers are organized in a hierarchical manner, with each layer learning progressively more abstract features from the input data. The final layer produces an output that can be used for decision making or prediction.

One of the main advantages of deep learning is its ability to automatically extract features from raw data, eliminating the need for manual feature engineering. This makes it suitable for handling large and complex datasets.

Some common applications of deep learning include image and speech rec